In [2]:
from PIL import Image
import numpy as np
import cv2
from scipy import ndimage
import matplotlib.pyplot as plt
from tqdm import tqdm
import imutils
from imutils.perspective import four_point_transform

In [7]:
def dilate_cv2(image, iterations=1):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.dilate(image, kernel, iterations=iterations)
 
 
def find_rects(contours):
    main_contours = []
    for contour in contours:
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
 
        area = int(rect[1][0] * rect[1][1])
        if area > 500:
            main_contours.append(box)
    return np.asarray(main_contours)
 
 
def find_edges_by_cv2(image, is_gray=True):
    if not is_gray:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    image = cv2.Canny(image, 75, 200)
    return image
 
 
def thresholding_otsu_cv2(img, blur=True, inverse=True, to_grayscale=True):
    orig_height = img.shape[0]
    img = imutils.resize(img, height=500)
    if blur:
        img = cv2.medianBlur(img, 5)
    if to_grayscale:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if inverse:
        ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    else:
        ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    img = imutils.resize(img, height=orig_height)
    return img
 
 
def remove_noise_cv2(img, iterations=2):
    kernel = np.ones((3, 3), np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=iterations)
    kernel = np.ones((3, 3), np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=iterations)
    return img
 
 
def find_docs_cv2(image):
    orig_image = image.copy()
    ratio = orig_image.shape[0] / 500.0
    image = imutils.resize(image, height=500)
 
    image = cv2.pyrMeanShiftFiltering(image, 21, 51)
    image = thresholding_otsu_cv2(image)
    image = remove_noise_cv2(image)
    image = dilate_cv2(image, 1)
    image = find_edges_by_cv2(image)
 
    contours = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
    contours = imutils.grab_contours(contours)
    main_contours = find_rects(contours) * ratio
 
    images = []
    for contour in main_contours:
        points = np.asarray(contour).reshape(4, 2)
        document = four_point_transform(orig_image.copy(), points)
        images.append(document)
    return main_contours

In [8]:
base = cv2.imread('out15.jpg')
find_docs_cv2(base)

array([], dtype=float64)